<a href="https://colab.research.google.com/github/Aniket21628/Pokemon_Chatbot_RAG/blob/main/Pokemon_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q langchain langchain-nvidia-ai-endpoints

In [3]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color = "#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from google.colab import userdata

api_key = userdata.get('NVIDIA_API_KEY')

In [5]:
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface = "State: "):
  def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
  return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [ ]:
pip install langchain-community wikipedia

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from datetime import datetime, timedelta
import os
import json

docs = WikipediaLoader(query="Pokemon and Everything about Pokemon", load_max_docs=1000).load()


In [11]:
def create_chunks_with_headers(doc, doc_index):
    chunk_size = 800
    chunk_overlap = 100
    chunks = []
    start = 0
    doc_content = doc.page_content
    doc_length = len(doc.page_content)

    while start < doc_length:
        end = min(start + chunk_size, doc_length)
        chunk = doc_content[start:end]

        if start != 0:
            chunk = doc_content[max(start - chunk_overlap, 0):end]

        chunk_json = {
            "meta_data": {
                "title": doc.metadata["title"],
                "summary": doc.metadata['summary'],
                "source_url": doc.metadata['source'],
            },
            "chunk_index": len(chunks) + 1,
            "content": chunk
        }
        chunks.append(chunk_json)

        start += chunk_size

    return chunks

In [ ]:
all_chunks = []

for i,doc in enumerate(docs):
  chunks = create_chunks_with_headers(doc, i+1)
  all_chunks.extend(chunks)
  print(f"Created {len(chunks)} chunks for document {i+1}")

print("All data has been processed")

In [ ]:
%pip install --upgrade --quiet lark langchain-chroma
%pip install --upgrade --quiet langchain-cohere